In [1]:
!pip install torch transformers flask beautifulsoup4 requests nltk

In [2]:
pip install accelerate -U

  Using cached accelerate-0.29.3-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.1
    Uninstalling accelerate-0.29.1:
      Successfully uninstalled accelerate-0.29.1
  Rolling back uninstall of accelerate
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-config.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-pfn7e1kr\accelerate-config.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-estimate-memory.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-pfn7e1kr\accelerate-estimate-memory.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate-launch.exe
   from C:\Users\WinTo\AppData\Local\Temp\pip-uninstall-pfn7e1kr\accelerate-launch.exe
  Moving to c:\users\winto\appdata\roaming\python\python311\scripts\accelerate.exe
   from C:\Users\WinTo\AppData\Loc

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\accelerate.exe' -> 'c:\\Python311\\Scripts\\accelerate.exe.deleteme'



In [3]:
from transformers import GPT2TokenizerFast, GPTNeoForCausalLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import torch

In [4]:
# Tokenizer and model setup
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125M')
tokenizer.pad_token = tokenizer.eos_token

In [5]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, inputs, outputs, max_length=512):
        self.tokenizer = tokenizer
        self.inputs = inputs
        self.outputs = outputs
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]
        
        input_encodings = tokenizer(input_text, padding="max_length", truncation=True, max_length=self.max_length)
        output_encodings = tokenizer(output_text, padding="max_length", truncation=True, max_length=self.max_length)
        
        return {
            "input_ids": torch.tensor(input_encodings['input_ids']),
            "attention_mask": torch.tensor(input_encodings['attention_mask']),
            "labels": torch.tensor(output_encodings['input_ids'])
        }
    

# Load and preprocess data
df = pd.read_json('Psychology-10K.json')
inputs = df['input'].tolist()
outputs = df['output'].tolist()
dataset = TextDataset(tokenizer, inputs, outputs)

In [6]:
# Training arguments and setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5, 
    logging_steps=2,
    warmup_steps=500,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


C:\Users\WinTo\AppData\Roaming\Python\Python311\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [7]:
# Train the model
trainer.train()

  0%|          | 0/3940 [00:00<?, ?it/s]

{'loss': 8.41, 'grad_norm': 280.4646911621094, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.0}
{'loss': 7.4749, 'grad_norm': 339.814697265625, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.0}
{'loss': 8.1241, 'grad_norm': 190.88009643554688, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.0}
{'loss': 8.219, 'grad_norm': 399.14837646484375, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.0}
{'loss': 8.0518, 'grad_norm': 347.3293762207031, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 8.3048, 'grad_norm': 360.0975646972656, 'learning_rate': 1.2000000000000002e-06, 'epoch': 0.01}
{'loss': 7.3189, 'grad_norm': 386.5830383300781, 'learning_rate': 1.4000000000000001e-06, 'epoch': 0.01}
{'loss': 8.004, 'grad_norm': 532.4410400390625, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.01}
{'loss': 7.0903, 'grad_norm': 566.72119140625, 'learning_rate': 1.8e-06, 'epoch': 0.01}
{'loss': 8.6519, 'grad_norm': 569.9920043945312, 'learning_rate': 2.0000000000000

TrainOutput(global_step=3940, training_loss=2.48259340836312, metrics={'train_runtime': 1332.0734, 'train_samples_per_second': 14.783, 'train_steps_per_second': 2.958, 'train_loss': 2.48259340836312, 'epoch': 2.0})

In [9]:
def predict_response(input_text, model, tokenizer):
    # Preprocess and tokenize the input text
    encoded_input = tokenizer.encode_plus(
        input_text,
        max_length=512,  # Ensure the length matches what was used during training
        truncation=True,
        padding='max_length',
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Move the tensors to the same device as the model
    encoded_input = {key: tensor.to(model.device) for key, tensor in encoded_input.items()}

    # Make prediction
    with torch.no_grad():
        output = model(**encoded_input)

    # Retrieve the probabilities (logits) and find the predicted class
    logits = output.logits
    predicted_class_index = logits.argmax(-1).item()

    # If you used LabelEncoder during preprocessing, you need to reverse that encoding:
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_label

# Example usage
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Make sure the model is on the right device
input_text = "i feel lonely"
response = predict_response(input_text, model, tokenizer)
print("Predicted Response:", response)

RuntimeError: a Tensor with 512 elements cannot be converted to Scalar